In [322]:
import numpy as np
import pandas as pd
import pyspark

In [323]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf # @udf("integer") def myfunc(x,y): return x - y
from pyspark.sql import functions as F # stddev format_number date_format, dayofyear, when
from pyspark.sql.types import StructField, StringType, IntegerType, StructType


In [324]:
print([(x.__name__,x.__version__) for x in [np, pd, pyspark]])

spark = pyspark.sql.SparkSession.builder.appName('stroke').getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
sc.setLogLevel("INFO")

[('numpy', '1.22.3'), ('pandas', '1.4.0'), ('pyspark', '3.2.1')]


In [325]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer,
                               OneHotEncoder, StringIndexer)


In [326]:
df = spark.read.csv('healthcare-dataset-stroke-data.csv',header=True,inferSchema=True)
print(df.count())
df.show()

5110
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
| 9046|  Male|67.0|           0|            1|         Yes|      Private|         Urban|           228.69|36.6|formerly smoked|     1|
|51676|Female|61.0|           0|            0|         Yes|Self-employed|         Rural|           202.21| N/A|   never smoked|     1|
|31112|  Male|80.0|           0|            1|         Yes|      Private|         Rural|           105.92|32.5|   never smoked|     1|
|60182|Female|49.0|           0|            0|         Yes|      Private|         Urban|           171.23|34.4|         smokes|     1|
| 1665|Female|79.0|           1|            0|    

In [327]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: string (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [328]:
from pyspark.sql.functions import col,sum
df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).show()

+---+------+---+------------+-------------+------------+---------+--------------+-----------------+---+--------------+------+
| id|gender|age|hypertension|heart_disease|ever_married|work_type|Residence_type|avg_glucose_level|bmi|smoking_status|stroke|
+---+------+---+------------+-------------+------------+---------+--------------+-----------------+---+--------------+------+
|  0|     0|  0|           0|            0|           0|        0|             0|                0|  0|             0|     0|
+---+------+---+------------+-------------+------------+---------+--------------+-----------------+---+--------------+------+



In [329]:
df.groupBy('bmi').count().show()

+----+-----+
| bmi|count|
+----+-----+
|34.4|   18|
|47.5|    3|
|20.5|   18|
|38.3|    2|
|45.4|    4|
|38.5|    7|
|26.5|   30|
|  51|    1|
|48.1|    1|
|29.4|   30|
|16.6|    8|
|14.2|    4|
|12.8|    1|
|26.7|   37|
|17.1|   12|
|36.1|    7|
|30.1|   26|
|40.1|   10|
|  54|    1|
|  15|    2|
+----+-----+
only showing top 20 rows



In [330]:
df.agg({'bmi': 'mean'}).show()

+------------------+
|          avg(bmi)|
+------------------+
|28.893236911794673|
+------------------+



In [331]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import when
df.withColumn('bmi', 
    when(df.bmi.endswith('N/A'),regexp_replace(df.bmi,'N/A','29')) \
   .otherwise(df.bmi)) \
   .show(truncate=False)

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|id   |gender|age |hypertension|heart_disease|ever_married|work_type    |Residence_type|avg_glucose_level|bmi |smoking_status |stroke|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|9046 |Male  |67.0|0           |1            |Yes         |Private      |Urban         |228.69           |36.6|formerly smoked|1     |
|51676|Female|61.0|0           |0            |Yes         |Self-employed|Rural         |202.21           |29  |never smoked   |1     |
|31112|Male  |80.0|0           |1            |Yes         |Private      |Rural         |105.92           |32.5|never smoked   |1     |
|60182|Female|49.0|0           |0            |Yes         |Private      |Urban         |171.23           |34.4|smokes         |1     |
|1665 |Female|79.0|1           |0            |Yes      

In [332]:
df.groupBy('stroke').count().show()

+------+-----+
|stroke|count|
+------+-----+
|     1|  249|
|     0| 4861|
+------+-----+



In [333]:
df_a = df.filter(df['stroke'] == 0)
df_b = df.filter(df['stroke'] == 1)

a_count = df_a.count()
b_count = df_b.count() 
ratio = a_count / b_count

df_b_oversampled = df_b.sample(withReplacement=True, fraction=ratio, seed=1)
df = df_a.unionAll(df_b_oversampled)

In [334]:
df.show()

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|30669|  Male| 3.0|           0|            0|          No|     children|         Rural|            95.12|  18|        Unknown|     0|
|30468|  Male|58.0|           1|            0|         Yes|      Private|         Urban|            87.96|39.2|   never smoked|     0|
|16523|Female| 8.0|           0|            0|          No|      Private|         Urban|           110.89|17.6|        Unknown|     0|
|56543|Female|70.0|           0|            0|         Yes|      Private|         Rural|            69.04|35.9|formerly smoked|     0|
|46136|  Male|14.0|           0|            0|         

In [335]:
df.groupBy('stroke').count().show()

+------+-----+
|stroke|count|
+------+-----+
|     0| 4861|
|     1| 4710|
+------+-----+



In [336]:
df.groupBy('bmi').count().show()

+----+-----+
| bmi|count|
+----+-----+
|20.5|   34|
|34.4|   54|
|38.3|    2|
|45.4|    4|
|47.5|   24|
|38.5|    7|
|  51|    1|
|26.5|   41|
|48.1|    1|
|29.4|   46|
|16.6|    8|
|14.2|    4|
|12.8|    1|
|26.7|   54|
|17.1|   12|
|36.1|    7|
|30.1|   26|
|40.1|   10|
|  54|    1|
|  15|    2|
+----+-----+
only showing top 20 rows



In [337]:
df=df.drop('id')

In [338]:
df.show()

+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|  Male| 3.0|           0|            0|          No|     children|         Rural|            95.12|  18|        Unknown|     0|
|  Male|58.0|           1|            0|         Yes|      Private|         Urban|            87.96|39.2|   never smoked|     0|
|Female| 8.0|           0|            0|          No|      Private|         Urban|           110.89|17.6|        Unknown|     0|
|Female|70.0|           0|            0|         Yes|      Private|         Rural|            69.04|35.9|formerly smoked|     0|
|  Male|14.0|           0|            0|          No| Never_worked|         Rural|           161.

In [339]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer,
                               OneHotEncoder, StringIndexer)

In [340]:
gender_indexer = StringIndexer(inputCol='gender', outputCol='gender_index')

gender_encoder = OneHotEncoder(inputCol='gender_index', outputCol='gender_vec')

In [341]:
marital_indexer = StringIndexer(inputCol='ever_married', outputCol='ever_married_index')

marital_encoder = OneHotEncoder(inputCol='ever_married_index', outputCol='ever_married_vec')

In [342]:
work_indexer = StringIndexer(inputCol='work_type', outputCol='work_type_index')

work_encoder = OneHotEncoder(inputCol='work_type_index', outputCol='work_type_vec')

In [343]:
Residence_indexer = StringIndexer(inputCol='Residence_type', outputCol='Residence_type_index')

Residence_encoder = OneHotEncoder(inputCol='Residence_type_index', outputCol='Residence_type_vec')

In [344]:
smoking_indexer = StringIndexer(inputCol='smoking_status', outputCol='smoking_status_index')

smoking_encoder = OneHotEncoder(inputCol='smoking_status_index', outputCol='smoking_status_vec')

In [345]:

#df = df.withColumn("bmi", df["bmi"].cast(IntegerType()))
df=df.drop("bmi")

In [346]:
df.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [347]:
df.columns

['gender',
 'age',
 'hypertension',
 'heart_disease',
 'ever_married',
 'work_type',
 'Residence_type',
 'avg_glucose_level',
 'smoking_status',
 'stroke']

In [348]:
assembler = VectorAssembler(inputCols=["gender_vec","age","hypertension","heart_disease","ever_married_vec","work_type_vec","Residence_type_vec","avg_glucose_level","smoking_status_vec"],
                           outputCol="features")

In [349]:
from pyspark.ml.classification import LogisticRegression



In [350]:
lr = LogisticRegression(featuresCol='features',labelCol='stroke')

In [351]:
from pyspark.ml import Pipeline

In [352]:
pipeline = Pipeline(stages=[gender_indexer,marital_indexer,work_indexer,Residence_indexer,smoking_status_indexer,
                            gender_encoder,marital_encoder,work_encoder,Residence_encoder,smoking_status_encoder,
                            assembler, lr])

In [353]:
train, test = df.randomSplit([0.7, 0.3])

In [354]:
lr_model = pipeline.fit(train)

In [356]:
results = lr_model.transform(test)
results['stroke','prediction'].show()

+------+----------+
|stroke|prediction|
+------+----------+
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
+------+----------+
only showing top 20 rows



In [358]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [361]:
lr_evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='stroke')

In [363]:
auc=lr_evaluator.evaluate(results)

Py4JJavaError: An error occurred while calling o8733.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 693.0 failed 1 times, most recent failure: Lost task 0.0 in stage 693.0 (TID 797) (DESKTOP-0UH1SHS executor driver): org.apache.spark.SparkException: Failed to execute user defined function (StringIndexerModel$$Lambda$3816/0x00000001015f1840: (string) => double)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:136)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage3.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:759)
	at scala.collectio

In [364]:
from pyspark.ml.classification import DecisionTreeClassifier